In [12]:
import os
import pandas as pd

# Define the main directory
main_dir = os.path.join(os.getcwd(), 'Run1')

# Initialize an empty list to store the rows
data = []

# Iterate through each folder in the main directory
for folder_name in os.listdir(main_dir):
    folder_path = os.path.join(main_dir, folder_name)
    if os.path.isdir(folder_path):
        # Find the positions of the first and last underscores
        first_underscore = folder_name.index('_')
        last_underscore = folder_name.rindex('_')
        
        # Extract TIME, INSTANCE, and MODE
        TIME = folder_name[:first_underscore]
        INSTANCE = folder_name[first_underscore + 1:last_underscore]
        MODE = folder_name[last_underscore + 1:]
        
        # Initialize GAP and TIME with default values
        GAP = 0
        SOLVING_TIME = 3700
        
        # Check for result.txt and extract GAP
        result_file_path = os.path.join(folder_path, 'results.txt')
        if os.path.exists(result_file_path):
            with open(result_file_path, 'r') as result_file:
                for line in result_file:
                    if line.startswith("GapClosed:"):
                        GAP = float(line.split(':')[1].strip())
                        break
        
        # Check for solving_statistic.txt and extract TIME
        solving_stat_file_path = os.path.join(folder_path, 'solving_statistic.txt')
        if os.path.exists(solving_stat_file_path):
            with open(solving_stat_file_path, 'r') as solving_stat_file:
                for line in solving_stat_file:
                    if line.startswith("Total Time"):
                        SOLVING_TIME = float(line.split(':')[1].strip())
                        break
        
        # Append the extracted data to the list
        data.append([INSTANCE, MODE, GAP, SOLVING_TIME])

# Create a pandas DataFrame from the collected data
df = pd.DataFrame(data, columns=['INSTANCE', 'MODE', 'GAP', 'TIME'])


In [13]:
df

,INSTANCE,MODE,GAP,TIME
0,ns1208400,vpc,0.000000,2877.51
1,traininstance6,gomory,0.000000,4.94
2,neos-1582420,gomory,0.053285,2.42
3,neos17,vpc,0.000000,0.10
4,30n20b8,gomory,0.096826,26.99
...,...,...,...,...
79,ns1830653,vpc,0.022232,98.72
80,graph20-20-1rand,gomory,0.000000,0.39
81,neos5,gomory,0.115741,0.02
82,neos-3216931-puriri,gomory,0.016841,38.83


In [14]:
# Pivot the DataFrame to the desired format
df_pivot = df.pivot(index='INSTANCE', columns='MODE', values=['GAP', 'TIME'])

# Flatten the multi-level columns
df_pivot.columns = ['_'.join(col).strip() for col in df_pivot.columns.values]
df_pivot.reset_index(inplace=True)

# Rename columns to the desired format
df_pivot.columns = ['INSTANCE', 'gomory_gap', 'vpc_gap', 'gomory_time', 'vpc_time']

In [15]:
df_pivot

,INSTANCE,gomory_gap,vpc_gap,gomory_time,vpc_time
0,30n20b8,9.682640e-02,0.000000e+00,26.99,3657.02
1,beasleyC3,1.179610e-02,1.179610e-02,0.07,129.94
2,binkar10_1,6.759320e-02,6.691340e-02,0.09,41.19
3,bnatt400,0.000000e+00,0.000000e+00,1.93,3531.65
4,cod105,8.656170e-03,4.375580e-03,4.13,163.12
5,cost266-UUE,2.727120e-01,1.863410e-01,0.37,298.08
6,csched007,8.924880e-02,3.389650e-02,0.25,237.80
7,exp-1-500-5-5,2.316010e-01,2.173120e-01,0.06,36.20
8,gen-ip002,2.148250e-02,9.064810e-03,0.01,0.17
9,glass4,0.000000e+00,-8.881780e-15,0.05,2.67


In [16]:
df_rounded = df_pivot.copy()
df_rounded['gomory_gap'] = (df_rounded['gomory_gap'] * 100).round(2)
df_rounded['vpc_gap'] = (df_rounded['vpc_gap'] * 100).round(2)

In [17]:
df_rounded

,INSTANCE,gomory_gap,vpc_gap,gomory_time,vpc_time
0,30n20b8,9.68,0.00,26.99,3657.02
1,beasleyC3,1.18,1.18,0.07,129.94
2,binkar10_1,6.76,6.69,0.09,41.19
3,bnatt400,0.00,0.00,1.93,3531.65
4,cod105,0.87,0.44,4.13,163.12
5,cost266-UUE,27.27,18.63,0.37,298.08
6,csched007,8.92,3.39,0.25,237.80
7,exp-1-500-5-5,23.16,21.73,0.06,36.20
8,gen-ip002,2.15,0.91,0.01,0.17
9,glass4,0.00,-0.00,0.05,2.67
